In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
from scipy.special import sph_harm 
from scipy import integrate
from scipy.integrate import dblquad
from scipy.integrate import quad

The equation used for projection of the spherical harmonics: 
$$
\begin{equation}
    \psi_{nm}=\int ^{2\pi}_0 \int^\pi_0 \psi(\theta, \phi)(Y^m_n(\theta,\phi))^\ast \sin \theta d\theta d\phi
\end{equation}
$$

In [2]:
def projection(f_tp, theta, phi, lMax): # Projection onto sph_harm: takes f(theta,phi) as input and returns f_{nm}
    t, p = np.meshgrid (theta, phi)
    f_nm = np.zeros((lMax, lMax), dtype=complex)
    for m in range (lMax): 
        for n in range (m, lMax):
            Y_nm = sph_harm (m, n, p.reshape(-1), t.reshape(-1))
            Y_ast = np.conj(Y_nm.reshape(N_p, N_t))
            f_nm[n,m] = integrate.simpson(integrate.simpson(f_tp*Y_ast*np.sin(t), theta), phi)
    return f_nm

The equation used for reconstruction:
$$
\begin{equation}
    \psi (\theta, \phi) = \sum ^{N}_{\lvert m \rvert = 0} \sum ^{N}_{n=\lvert m \rvert} \psi_{nm} Y^m_n(\theta, \phi)
\end{equation}
$$

In [4]:
def reconstruction(f_nm, theta, phi, lMax):  #Reconstruction from sph_harm, takes f_{nm} and returns f(theta,phi)
    f_TP = np.zeros((N_p, N_t))
    #double sum
    for m in range(lMax):
        for n in range(m, lMax):
            Y_nm = sph_harm (m, n, p.reshape(-1), t.reshape(-1)).reshape(N_p, N_t)
            if m==0:
                f_TP += np.real(f_nm[n,m]*Y_nm) # A+=B is equivalent to A=A+B
            else:
                f_TP += 2.*np.real(f_nm[n,m]*Y_nm) #the negative value need to also be accounted for
        
    return f_TP